In [1]:
import cptac
import scipy
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
import statistics
import parse_correlations_dataframe as get_corr
import copy

In [6]:
cptac.download("endometrial")
en = cptac.Endometrial()

In [39]:
def find_mut_tumor(cancer_type, gene):
    gene_multi = cancer_type.multi_join({'proteomics': gene, 'transcriptomics': gene, 'somatic_mutation': gene}, tissue_type = 'tumor')
    normal = cancer_type.multi_join({'proteomics': gene, 'transcriptomics': gene}, tissue_type= 'normal', flatten = True)
    normal.columns = ['proteomics', 'transcriptomics']
    trans = list(gene_multi[str(gene +"_transcriptomics")])
    prot = list(gene_multi[str(gene + '_proteomics')])
    group = []
    for i in gene_multi[str(gene + '_Mutation_Status')]:
        if type(i) == str:
            group.append("mutation")
        else:
            group.append("wt") 
    group.extend(['normal']*len(normal))
    prot.extend(list(normal['proteomics']))
    trans.extend(list(normal['transcriptomics']))
    gene_df = pd.DataFrame({'Type': group, 'Proteomics': prot, 'Transcriptomics': trans})
    gene_df = gene_df.dropna()
    return gene_df

In [40]:
def get_df_with_type(df,input_type):
    return(df.loc[df['Type']==input_type])

In [41]:
df = find_mut_tumor(en, 'TP53')

cptac warning: You tried to flatten a column index that didn't have multiple levels, so we didn't actually change anything. (/Users/benkk/anaconda3/envs/cptac_dev/lib/python3.7/site-packages/cptac/dataset.py, line 697)


In [42]:
df

,Type,Proteomics,Transcriptomics
0,mutation,0.2950,11.62
1,wt,0.2770,11.83
2,wt,-0.8710,10.73
3,wt,-0.3430,10.95
4,mutation,3.0100,10.69
...,...,...,...
115,normal,0.7950,9.83
116,normal,0.6270,10.64
119,normal,-0.2370,9.62
120,normal,-0.7290,10.45


In [43]:
df2 = get_df_with_type(df,"normal")

In [44]:
df2

,Type,Proteomics,Transcriptomics
95,normal,-0.0919,9.87
96,normal,-0.5930,10.52
98,normal,-0.1310,9.98
99,normal,-0.0844,9.92
113,normal,0.5140,10.35
114,normal,0.2500,10.27
115,normal,0.7950,9.83
116,normal,0.6270,10.64
119,normal,-0.2370,9.62
120,normal,-0.7290,10.45


In [45]:
df2.corr(method="pearson")

,Proteomics,Transcriptomics
Proteomics,1.000000,-0.045068
Transcriptomics,-0.045068,1.000000
